In [383]:
import numpy as np
from scipy import optimize
from scipy.optimize import Bounds, LinearConstraint
import plotly.express as px
import matplotlib.pyplot as plt
from sympy import symbols, solve
import pandas as pd
from scipy.stats import beta
from scipy.integrate import quad

In [312]:
def q_star(x=np.linspace(0.1,0.9), y=1, p=0.2):
    
    """
    
    This function returns the optimal coverage amount q
    given the values; x, y, p.
    
    ARGUMENTS:
    x (monetary loss): can be a 1d array
    y (value of assets in some numeraire): can be a float or int
    p (probability of observing the loss x): can be a float between 0 <= p <= 1
    
    """
    
    # Define utility function given parameter; v = -2
    def u(z, v=-2):
        return (z**(1+v)) / (1+v)
    
    # Define expected utility function with free parameters q and x
    def V(q, x):
        return p * u(y - x + q - p*q) + (1 - p) * u(y - p*q)

    # initial value of q
    q_guess = 0
    
    # Optimize any q given constraint 0 <= q <= x for x in [0.1; 0.9] and q in [0;x]
    result = [] # Empty list to store results
    for i in x: # Loop through the different values of x
        
        # Define bounds for q
        bnds = ((0,i),)
        
        # Define objective function to be optimized, with free parameter q 
        obj_func = lambda q: -V(q, x=i)
        
        # Optimize obj func. 
        res = optimize.minimize(obj_func, q_guess, bounds = bnds)
        
        # Append optimal value q^star_i given 0.1 <= x_i <= 0.9
        result.append(round(res.x[0],7))
        
    
    # Return list containing different optimal values of q given value of x
    return result

In [313]:
## Plot q_star given x in range [0.1, 0.9]
y_axis = q_star()
x_axis = np.linspace(0.1,0.9)

fig = px.line(x = x_axis, y = y_axis, title='Optimal value of q given x',
             labels=dict(x ="$x$", y ="$q^\star$"))
fig.show()



In [355]:
def pi_opt(x=0.6, y=1, p=0.2):
    
    """
    
    """
    
    # Define utility function given parameter; v = -2
    def u(z, v=-2):
        return z ** (1+v) / (1+v)

    # Utility with insurance
    def V(pi, q):
        return p * u(y - x + q - pi) + (1 - p) * u(y - pi) - V_0

    # Set initial value of pi for maximization
    pi_guess = 0
    
    # Utility without insurance
    V_0 = p * u(y - x) + (1 - p) * u(y)
    
    # Optimize any pi_thilde given q in [0.1; 0.6]
    result = [] # Empty list to store results
    for i in np.linspace(0.1, 0.6): # Loop through the different values of q
        
        # Find pi thilde
        pi_thilde = optimize.root(V, pi_guess, args = (i))
        
        """
    
        # Define symbol "pi"
        pi = symbols('pi')
        
        # Objective function to solve
        obj_func = V(pi) - V_0

        # Derive phi_thilde
        pi_thilde = solve(obj_func)
        
        """
        
        # Append results to empty list
        #result.append(float(round(pi_thilde[0],5)))
        result.append(pi_thilde.x)
    
    # Return list containing different optimal values of q given value of x
    return result  

In [356]:
pi_opt()

ValueError: Integers to negative integer powers are not allowed.

In [297]:
## Plot pi_thilde given q in range [0.1, 0.6]
y_axis = np.array(pi_opt())
x_axis = np.linspace(0.1, 0.6)
fig2 = px.line(x = x_axis, y = y_axis, title = 'Optimal value of pi given q',
              labels=dict(x ="$q$", y ="$\pi$"))
fig2.show()

ValueError: Integers to negative integer powers are not allowed.

In [243]:
## Plot pi_thilde given q in range [0.1, 0.6] and pi[q,p]

# Define pi[q,p]
pi_pq = 0.2*np.linspace(0.1, 0.6)

# Create DF for plotting
df = pd.DataFrame(pi_pq, columns = ['$\pi[p;q]$'])
df['$\pi_{thilde}$'] = pi_opt()

In [254]:
fig3 = px.line(df)
fig3.show()

In [406]:
gamma = np.linspace(0,1)
q = np.random.beta(2,7)*gamma

def f():
    return beta.pdf(np.random.beta(2,7), 2, 7)

def u(z):
    return z**(-1) / -1

In [381]:
quad(u(1-()), 0, 1)

1.0450944000000002


In [455]:
def integral(gamma, pi):
    
    np.random.seed(2)
    
    result = []
    for i in range(10000):
        
        x = np.random.beta(2, 7)
        f_x = beta.pdf(x, 2, 7)

        obj_func = u(1-(1-gamma)*x-pi)*f_x

        result.append(obj_func)

    return np.array(result).mean()

In [456]:
integral(0.45,0.1)

-2.887392732151206

In [457]:
integral(0.9,0.2)

-2.930543989299542